In [39]:
import sh
from toolz import thread_first, compose
from operator import getitem

def first(x):
    return x[0]

def get_matching_processes(pattern,m=sh):
    all_processes = m.ps('-A')
    target_processes = m.grep(all_processes,pattern, _ok_code = [0,1]).split('\n')[:-1]
    target_pids = map(compose(int,first,str.split,str),target_processes)
    return target_pids

def kill_pids(pids,m=sh):
    for pid in pids:
        m.kill(pid)
        
def kill_matching_processes(pattern,m=sh):
    pids = get_matching_processes(pattern,m)
    kill_pids(pids,m)        
    
def pull_latest_code(path,m=sh):
    m.git('fetch','--all')
    m.git('clean','-df')
    m.git('reset','--hard','origin/master')
    
def restart_ipython_server(m):
    m.nohup('ipython notebook','&')    

In [14]:
# update_server.py
kill_matching_processes("ipython",server)
pull_latest_code(path,server)
restart_ipython_server(server)

In [23]:
# deploy.py
update_path = "/home/ubuntu/prod/assay-explorer/update_server.py"
server = sh.ssh.bake('-i','../my-pair.pem','ubuntu@52.8.164.123')
server.python(update_path)

ErrorReturnCode_128: 

  RAN: '/usr/bin/ssh -i ../my-pair.pem ubuntu@52.8.164.123 git fetch --all'

  STDOUT:


  STDERR:
fatal: Not a git repository (or any of the parent directories): .git
